# Recommendation system using NLP (Persian movies)

This documentation was entirely written in English for other presentation purposes!!!

Note: (با سلام استاد امینی، من به همراه این فایل یک یا چند فایل صوتی هم ارسال می‌کنم که هر قسمت را به صورت جداگانه توضیح می‌دهم.)

# Imports

In [1]:
import pandas as pd
import numpy as np
import re
from hazm import Stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from hazm import Lemmatizer

# Dataset imports
In addition to the main dataset (Persian movies) we need to import Pesian stop words as well.
(stop words contain words that have 0 weight. in other words, we need to remove them since they do not have any acctual value or meaning. Examples: ( را, باید, رفت و ...)

In [2]:
df = pd.read_csv('dataset.csv')
stop_words = pd.read_csv('all_words.txt')

# We need to convert it into a list for better compatibility with re library

stop_words_list = stop_words['words'].tolist()

pd.set_option('display.max_colwidth', None)

In [3]:
df.head()

,Link,EN_title,PENGLISH_title,PERSIAN_title,Content_1,Content_2,Score,Year,Genre,Time
0,https://www.imvbox.com/watch-persian-movie-iranian-movies/local-anaesthetic-bi-hessie-mozeie,Local Anaesthetic,Bi Hessie Mozeie,بی‌حسی موضعی,جلال‌، دانشجوی سابق رشته فلسفه، متوجه می‌شود خواهرش که به اختلال روانی دوقطبی مبتلاست، با مردی ثروتمند به نام شاهرخ ازدواج کرده که به شرط‌‌بندی بازی فوتبال اعتیاد دارد. جلال با عصبانیت از خانه بیرون می‌زند تا به خانه دوستش، بهمن، برود که یک آهنگ‌ساز زیرزمینی است. در مسیر جلال با یک راننده تاکسی به نام ناصر آشنا می‌شود و این دو شب عجیبی را در کنار هم سپری می‌کنند.,"Jalal, a dropouts philosophy student, realizes that her sister Mary, who has a Bipolar disease, is married to a bourgeois man named Shahrokh who is addicted to football betting. He left the house angrily, and goes to his friend's house, Bahman, who is an underground composer. He meets a strange taxi driver named Nasser on his way. They spent a strange night together.",4.8,2018,Drama,73
1,https://www.imvbox.com/watch-persian-movie-iranian-movies/disturbance-ashoftegi,Disturbance,Ashoftegi,آشفته گی,«آشفته‌گی» رئالیستی و اجتماعی نیست. یک فیلم است درباره عشق و جنایت....,"After the murder of his rich twin brother, Barbod assumes his identity and deals with the problems that come with it.“Chaos” is not a realist or social movie. It is a film about love and crime.",3.8,2018,Crime,78
2,https://www.imvbox.com/watch-persian-movie-iranian-movies/highlight-haylayt,Highlight,Haylayt,هایلایت,یک تصادف اتومبیل آدم‌هایی را در تقابل با هم قرار می‌دهد. مراقبت از مصدومین و انتظار برای به هوش آمدن آنها انتظاری طولانی و جانکاه است.,"A man and a woman are have a car accident and go into a coma, but their spouses don't know why these two were together in the first place.A car accident causes many people to face each other. Nursing the injured and waiting for their recovery is a long and wearisome wait.",4.4,2017,Drama,77
3,https://www.imvbox.com/watch-persian-movie-iranian-movies/gilda-gilda,Gilda,Geelda,گیلدا,گیلدا ماجرای زنی به نام «گیلدا» را روایت می کند که صاحب رستوانیست. شبی اتفاقی برایش می افتد که سیاه ترین شب زندگی اش می شود و برای نجات زندگی اش تا صبح فرصت دارد.,Gilda who owns a restaurant has a terrible night. She has only till morning to save her life from death.Gilda is the owner of Sepid restaurant; she has overcome the darkest night of her life. The next morning is the latest opportunity to save her life.,3.8,2018,Drama,79
4,https://www.imvbox.com/watch-persian-movie-iranian-movies/atmosphere-station-istgha-e-atmosfer,Atmosphere Station,Istgahe Atmosfer,ایستگاه اتمسفر,این فیلم روایت گر داستان زندگی زوج جوانی به اسم «مرجان» و «وحید» است که هشت ماهی می‌شود طلاق گرفته‌اند اما به دلایل خاصی مجبورند به زندگی در کنار هم ادامه دهند. در این میان مرجان که تصمیم به مهاجرت دارد، تمام تلاشش را برای گرفتن اقامت دائم کانادا می‌کند اما …,"Vahid and Marjan are a young couple who have gotten divorced a few months ago without telling anybody. Marjan dies in an accident and Vahid is blamed for it. Later, it is revealed that Marjan has borrowed a lot of money from the neighbors. Vahid says he knows nothing about it, but …Atmosphere watches over earth and we watch over each other…",5.6,2017,Drama,85


In [4]:
stop_words

,words
0,فیلم
1,کن
2,کرد
3,کردن
4,باش
...,...
1555,ما رو
1556,رو
1557,داره
1558,این دفعه


# Preproccesing the data
some of the content needs to be processed such as null values in the Content_1 column, genre names and column types to prevent any errors later on. 

In [5]:
# Fill missing values
df['Content_1'] = df['Content_1'].fillna('')

# Convert genre to Persian
genre_mapping = {'Drama': 'دراما', 'Crime': 'جرم و جنایت جنایی', 'Human Interest & Society': 'ععلایق انسانی وجامعه',
                 'Comedy': 'کمدی',
                 'Adventure': 'ماجراجویی', 'Mystery': 'رازآلود', 'War': 'جنگ', 'Action': ' اکشن و هیجانی',
                 'Portrait': 'پورتریت',
                 'Experimental': 'آزمایشی', 'Romance': 'عاشقانه', 'Family': 'خانواده', 'History': 'تاریخی',
                 'Animation': 'کارتون انیمیشن',
                 'Culture & Traditions': 'فرهنگی ', 'Arts & Literature': 'هنری ادبی', 'Music': 'موزیکال',
                 'Architecture & Urbanism': 'معماری',
                 'Horror': 'ترسناک وحشت ناک', 'Nature & Wildlife': 'طبیعت حیت وحش', 'Thriller': 'دلهره آور هیجانی'}
df['Genre'] = df['Genre'].replace(genre_mapping)

# Convert columns to string type
df = df.astype(str)

# adding the genre column to the Content_1 column for more accuracy in the model
df['Content_1'] = df['Content_1'] + ' ' + df['Genre']

In [6]:
df.head()

,Link,EN_title,PENGLISH_title,PERSIAN_title,Content_1,Content_2,Score,Year,Genre,Time
0,https://www.imvbox.com/watch-persian-movie-iranian-movies/local-anaesthetic-bi-hessie-mozeie,Local Anaesthetic,Bi Hessie Mozeie,بی‌حسی موضعی,جلال‌، دانشجوی سابق رشته فلسفه، متوجه می‌شود خواهرش که به اختلال روانی دوقطبی مبتلاست، با مردی ثروتمند به نام شاهرخ ازدواج کرده که به شرط‌‌بندی بازی فوتبال اعتیاد دارد. جلال با عصبانیت از خانه بیرون می‌زند تا به خانه دوستش، بهمن، برود که یک آهنگ‌ساز زیرزمینی است. در مسیر جلال با یک راننده تاکسی به نام ناصر آشنا می‌شود و این دو شب عجیبی را در کنار هم سپری می‌کنند. دراما,"Jalal, a dropouts philosophy student, realizes that her sister Mary, who has a Bipolar disease, is married to a bourgeois man named Shahrokh who is addicted to football betting. He left the house angrily, and goes to his friend's house, Bahman, who is an underground composer. He meets a strange taxi driver named Nasser on his way. They spent a strange night together.",4.8,2018,دراما,73
1,https://www.imvbox.com/watch-persian-movie-iranian-movies/disturbance-ashoftegi,Disturbance,Ashoftegi,آشفته گی,«آشفته‌گی» رئالیستی و اجتماعی نیست. یک فیلم است درباره عشق و جنایت.... جرم و جنایت جنایی,"After the murder of his rich twin brother, Barbod assumes his identity and deals with the problems that come with it.“Chaos” is not a realist or social movie. It is a film about love and crime.",3.8,2018,جرم و جنایت جنایی,78
2,https://www.imvbox.com/watch-persian-movie-iranian-movies/highlight-haylayt,Highlight,Haylayt,هایلایت,یک تصادف اتومبیل آدم‌هایی را در تقابل با هم قرار می‌دهد. مراقبت از مصدومین و انتظار برای به هوش آمدن آنها انتظاری طولانی و جانکاه است. دراما,"A man and a woman are have a car accident and go into a coma, but their spouses don't know why these two were together in the first place.A car accident causes many people to face each other. Nursing the injured and waiting for their recovery is a long and wearisome wait.",4.4,2017,دراما,77
3,https://www.imvbox.com/watch-persian-movie-iranian-movies/gilda-gilda,Gilda,Geelda,گیلدا,گیلدا ماجرای زنی به نام «گیلدا» را روایت می کند که صاحب رستوانیست. شبی اتفاقی برایش می افتد که سیاه ترین شب زندگی اش می شود و برای نجات زندگی اش تا صبح فرصت دارد. دراما,Gilda who owns a restaurant has a terrible night. She has only till morning to save her life from death.Gilda is the owner of Sepid restaurant; she has overcome the darkest night of her life. The next morning is the latest opportunity to save her life.,3.8,2018,دراما,79
4,https://www.imvbox.com/watch-persian-movie-iranian-movies/atmosphere-station-istgha-e-atmosfer,Atmosphere Station,Istgahe Atmosfer,ایستگاه اتمسفر,این فیلم روایت گر داستان زندگی زوج جوانی به اسم «مرجان» و «وحید» است که هشت ماهی می‌شود طلاق گرفته‌اند اما به دلایل خاصی مجبورند به زندگی در کنار هم ادامه دهند. در این میان مرجان که تصمیم به مهاجرت دارد، تمام تلاشش را برای گرفتن اقامت دائم کانادا می‌کند اما … دراما,"Vahid and Marjan are a young couple who have gotten divorced a few months ago without telling anybody. Marjan dies in an accident and Vahid is blamed for it. Later, it is revealed that Marjan has borrowed a lot of money from the neighbors. Vahid says he knows nothing about it, but …Atmosphere watches over earth and we watch over each other…",5.6,2017,دراما,85


# CountVectorizer
We can find and add the most used words in our dataset using this method. Removing these words gives us a cleaner and more accurate results in the end.

In [7]:

# initialize CountVectorizer
vectorizer = CountVectorizer()

# fit and transform the text column
X = vectorizer.fit_transform(df['Content_1'])

# get the feature names
feature_names = vectorizer.get_feature_names_out()

# sum up the occurrences of each word
word_counts = X.sum(axis=0)

# convert the word counts to a 1D array
word_counts_array = np.squeeze(np.asarray(word_counts))

# create a DataFrame with word counts and their corresponding feature names
word_counts_df = pd.DataFrame({'Word': feature_names, 'Count': word_counts_array})

# sort the dataframe by word counts to see the most frequent words
most_common_words = word_counts_df.sort_values(by='Count', ascending=False)

# extract the top 30 most common words from the dataframe
top_common_words = most_common_words.head(30)['Word'].tolist()

# append the top common words to your existing stop words list
stop_words_list.extend(top_common_words)

most_common_words.head(30)


,Word,Count
1813,به,4007
7441,می,3745
3423,در,2912
9405,که,2758
631,از,2558
3875,را,2427
651,است,1855
1247,با,1725
1119,او,1524
5079,شود,1069


In [8]:
len(stop_words_list)

1590

# Text cleaning process
Using the re library we can remove stop words from our data in the Content_1 column and remove all of the special charachters as well. 

In [9]:
def remove_words_from_text(text):
    global stop_words_list
    # the line creates a regular pattern for our stop words and it only targets whole words
    pattern = '|'.join(r'\b{}\b'.format(re.escape(word)) for word in stop_words_list)
    # remove repetetive occurrences of  words in our column and it matches words regardless of their case
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    # removes special charachters that are not considered words
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    return cleaned_text

example_clean=df['Content_1'].apply(remove_words_from_text)
example_clean

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                جلال دانشجوی   فلسفه متوجه  خواهرش   اختلال روانی دوقطبی مبتلاست  مردی ثروتمند   شاهرخ ازدواج    شرط بازی فوتبال اعتیاد  جلال  عصبانیت    زند    دوستش بهمن برود   آهنگساز زیرزمینی   م

# Hazm library
Using the hazm library we are able to stemm (returning our words to their roots) our text inputs.

**note: some of the word forms might not make any sense (it might seem incorrect but it is actually stemmed in a way that is not too expensive for the computer) but it does not matter since all the words go through the stemming proccess and in the end these word forms match togher.

In [10]:
def apply_hazm_stemmer(text):
    stemmer = Stemmer()
    # removes any word with less than 3 letters
    stemmed_words = [stemmer.stem(word) for word in text.split() if len(word) > 3]
    return ' '.join(stemmed_words)

input_example='یک فیلمی احساسی و پلیسی'

apply_hazm_stemmer(input_example)

'فیلم احساس پلیس'

# Word2vec
Word2Vec is a technique used to convert words into numbers, so computers can understand them better. It does this by learning from a lot of text data and figuring out which words are similar based on how they're used in sentences.

Imagine you have a big collection of books. Word2Vec would go through all those books and learn that words like "king" and "queen" often appear together, so they must be related. Similarly, it would notice that "cat" and "dog" often appear in similar contexts, so they must be related too.

Once Word2Vec learns these relationships, it represents each word as a unique set of numbers (a vector) in a way that similar words have similar sets of numbers. This makes it easier for computers to understand and work with words in a meaningful way.

Word2Vec works by training a neural network model on a large corpus of text data. It learns to predict the context of words based on their surrounding words in sentences. There are two main architectures for Word2Vec:

Continuous Bag of Words (CBOW):

In CBOW, the model predicts the target word based on its context, which consists of the surrounding words in a sentence.
The input to the model is a context window of surrounding words, and the output is the target word.
CBOW is useful for small datasets and when the context of words is relatively stable.

Skip-gram:

In Skip-gram, the model predicts the surrounding words (context) given a target word.
The input to the model is a target word, and the output is the context window of surrounding words.
Skip-gram is useful for large datasets and capturing semantic relationships between words.

# Models usage in our case
Using this model we can extract all the similar words basesd on the context of our target word (in this case each and every word that our user gives us as  input).

In [11]:

def word2vec(column, cleaner, user_input, num_results=10):
    # Initialize Stemmer for Persian language
    stemmer = Stemmer()
    
    # Preprocess the text column using the provided cleaner function
    text = column.apply(lambda x: apply_hazm_stemmer(remove_words_from_text(x)))
    
    # Tokenize sentences into words
    tokenized_sentences = [word_tokenize(sentence) for sentence in text]
    
    # Train Word2Vec model on tokenized sentences
    model = Word2Vec(tokenized_sentences, vector_size=150, window=4, min_count=10, workers=4)

    # Initialize lists and dictionary for storing similar words
    sim_list = []
    sim_dic = {}

    # Clean and preprocess the user input
    user_input = cleaner(user_input)
    user_input = apply_hazm_stemmer(remove_words_from_text(user_input))

    # Append the preprocessed user input to the list of similar words
    sim_list.append(user_input)

    # Find similar words for each word in the user input
    for word in user_input.split():
        if len(word) > 3:
            try:
                # Get the most similar words from the Word2Vec model
                similar_words = model.wv.most_similar(word, topn=5)
                for sim_words, relevance in similar_words:
                    if sim_words not in sim_dic:
                        sim_dic[sim_words] = relevance
                    else:
                        sim_dic[sim_words] += relevance

            except KeyError:
                continue

    # Filter similar words based on relevance score
    print(sim_dic)
    sim_dic_copy = sim_dic.copy()

    for key, relevance in sim_dic_copy.items():
        if float(relevance) < 0.7:
            sim_dic.pop(key, None)

    # Append filtered similar words to the list
    for key, relevance in sim_dic.items():
        sim_list.append(key)

    # Join the list of similar words into a single string
    sim_string = ' '.join(sim_list)
    return sim_string


# Results
print(sim_dic): word 2 vec returns similar words +  their relevance score to the target word (0 to 1) since our  dataset is very  small the results are not as good as it can be.

In [12]:
word2vec(df['Content_1'],remove_words_from_text,'فیلم پلیسی')

{'همسر': 0.9997009038925171, 'موقع': 0.9996957182884216, 'موفق': 0.9996913075447083, 'ایر': 0.9996863603591919, 'خانواده': 0.9996771216392517}


'پلیس همسر موقع موفق ایر خانواده'

# TF_IDF model
TF-IDF, or Term Frequency-Inverse Document Frequency, is a technique used in natural language processing to evaluate the importance of words in documents within a collection. It calculates two values for each word:

1-Term Frequency (TF): Measures how often a word appears in a document.

2-Inverse Document Frequency (IDF): Measures how rare a word is across all documents in the collection.

TF-IDF combines these values to assign a score to each word, indicating its importance in a document relative to the entire collection. Words with higher TF-IDF scores are considered more significant to the document. It's commonly used in tasks like text classification and information retrieval.

# Our use for the model
Using the results from the previous model (word2vec) we will turn it into a vector. Additionally, we will turn all the content in our Content_1 column into vectors and in the end we can compare our input column with all our Content_1 vectors to  get the most accurate results.

In [20]:
def tf_idf(X_column, user_input, num_results=10):
    # Initialize TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    
    # Preprocess text data in the input column using stemming and word removal
    X = X_column.apply(lambda x: apply_hazm_stemmer(remove_words_from_text(x)))
    
    # Compute TF-IDF matrix for the preprocessed text data
    tfidf_matrix = tfidf_vectorizer.fit_transform(X)

    # Remove specified words from user input
    main_words = remove_words_from_text(user_input)
    
    # Get similar words using Word2Vec model
    similar_words_w2v = word2vec(X_column, remove_words_from_text, main_words)
    
    # Apply stemming to similar words obtained from Word2Vec
    similar_words_w2v = ' '.join([apply_hazm_stemmer(word) for word in similar_words_w2v.split()])

    # Preprocess user input by applying stemming and combining with similar words
    original_inp = remove_words_from_text(user_input)
    original_inp =' '.join([apply_hazm_stemmer(word) for word in original_inp.split()])
    original_inp += " "+ similar_words_w2v
    
    # Filter out short words
    original_inp = ' '.join([word for word  in original_inp.split() if len(word) > 3])
    print(original_inp)

    # Transform user input into TF-IDF vector
    user_tfidf = tfidf_vectorizer.transform([original_inp])
    
    # Compute cosine similarity between user input TF-IDF vector and corpus TF-IDF matrix
    similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    
    # Sort and get indices of the most similar documents
    similar_movies_indices = similarities.argsort()[:-num_results - 1:-1]

    # Extract specific columns from the DataFrame
    similar_movies = df.iloc[similar_movies_indices][['Content_1']] 
    
    # Return similar movies
    return similar_movies


In [24]:
user_input = "فیلم پلیسی"
recommended_movies = tf_idf(df['Content_1'], user_input)
recommended_movies

{'ایر': 0.9997029900550842, 'موقع': 0.9997016787528992, 'همسر': 0.9996955394744873, 'موفق': 0.9996926784515381, 'فرار': 0.9996867179870605}
پلیس پلیس موقع همسر موفق فرار


,Content_1
560,گناهکاران داستان «تدین» پلیس با سابقه و مسنی است که تا حدودی تند خو و کله شق است. وی بتازگی در پیگیری های خود مرتکب تخلف شده است؛ لذا رؤسای وی در اداره، در پرونده قتل دختری به نام «آسمان» یک پلیس جوان و حرفه ای را به عنوان مراقب و همکار با او همراه می کنند. جرم و جنایت جنایی
115,این فیلم داستان زنی به نام آذر است که از طریق دلالی و راه اناختن کار دیگران از طرق غیر قانونی پول در میآورد ولی سایه پلیس را هم همواره بر شانه اش دارد... جرم و جنایت جنایی
63,"این داستان زندگی سه زن در آستانه فروپاشی روحی و خود کشی است. لیلا که در فروشگاهی بزرگ صندوقدار است در راه بازگشت به خانه مورد تجاوز قرار گرفته و شکایتش به پلیس راه به جایی نمیبرد. وی پس از آنکه با خشونت برادر معترضش مواجه میشود به ناچار خانواده را ترک و سپس خود را با طناب دار میزند. ماریا پانزده روزی است که از همسر نامحبوبش ""ژوار"" خبری ندارد، تحقیقات پلیس ظن را به ماریا میبرد که ممکن است ژوار را سر به نیست کرده باشد که در این صورت سرنوشت غم باری انتظارش را میکشد. ""آنت"" که مدتی است دور از همسر و دخترش زندگی میکند و مراقب مادر بیمارش است، در پی ناراحتی های جسمی وقتی به پزشک مراجعه میکند آشکار میشود که گرفتار سرطان شده است. آنت به رغم امیدواری های دوستان ایرانی اش ، اما تصمیم خود را میگیرد و قصد خودکشی با اسلحه را میکند، اما پیش از آن در تصادف مرگبار اتومبیلش در میگذرد. دراما"
216,یک باند قاچاق بین المللی عتیقه وارد ایران می‌شوند و پلیس ایران آنان را ردیابی کرده و ماجراهایی اکشن را به دنبال می آورد اکشن و هیجانی
81,دو زندانی قصد دارند از زندان فرار کنند. کمدی
50,فیلم داستان سه مرد است که در اجتماع از موقعیت عالی و قدرت و اقتدار برخوردار بوده ولی در منزل از همسران خود بشدت حساب می برند تا اینکه روزی گیر کردن این سه مرد در آسانسور با یک خانم غریبه سرآغاز فرار از دست همسران خود می شود که به کمک یک گروگان گیر به جزیره کیش پناه می برند و کمدی
35,در یک ورزشگاه قتلی رخ داده و مظنون نیز مشخص است. چند مامور پلیس مشغول بررسی پرونده هستند و باید با توجه به سرنخ‌ها سیر وقوع قتل را پیدا کنند. جرم و جنایت جنایی
263,ماجرای این فیلم شرکتی را روایت می کند که تحت عنوان کمک به نخبگان در حال کلاهبرداری است. بعد از تحقیقات گسترده، پلیس به ماجرا پی برده و برای دستگیری اعضای این شرکت اقدام می کند تا اینکه در طول داستان اتفاقات مختلفی رقم می خورد کمدی
674,فیلم اسب حیوان نجیبی است ساخته عبدالرضا کاهانی و با بازی رضا عطاران داستان پلیسی است که سعی دارد از مردم به بهانه های مختلف اخاذی کند و هر کدام از مردم به شکلی مشغول انجام کاری خلاف قانون هستند... کمدی
290,“فرار از کمپ” در قسمت سوم و بعد از گذراندن فراز و نشیب های بسیار راه جدیدی را برای بازپروری معتادان پیدا می کند تا بلکه بتواند از این راه موفق شود اما… کمدی
